In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

# Assignment: Golf putting probability

In golf, the term [putt](https://en.wikipedia.org/wiki/Golf_swing#Putt) refers to a precision stroke that is executed by the player from a relatively short distance, aiming at sending the ball directly in the hole. 

Consider the following dataset of putt strokes executed by professional golf players, where $x$ is the distance from the hole discretized in 19 *levels*; $n$ is the total number of shots recorded at each distance level; and $y$ the number of successful putts, among the $n$ recorded.

In [ ]:
data = pd.read_csv("golf.csv")
data["x"] = data["x"] # normalized units. True distance (in feet) is 20*x
data["y"] = data["y"]
data["n"] = data["n"]
data.head()

Note that the distance $x$ is provided in 0-1 normalized units for numerical convenience. The true distance (in feet) may be obtained by multiplying $x$ by 20. Please stick with the normalized units in the assignment.

## Data exploration

We may compute and visualize the fraction of successful puts at all distance levels:

In [ ]:
x = data["x"].values
y = data["y"].values
n = data["n"].values
frac = y/n # fraction of successful puts at a given distance level

plt.figure()
plt.plot(x, frac, "*")
plt.xlabel("Distance from hole (normalized units)")
plt.ylabel("Fraction of successful putts (-)");

Clearly, the probability of making the shot decreases with the distance from the hole. We aim at building a probabilistic model that relates the probability of successful putts with the distance from the hole.

## Modeling assumptions

For the probabilistic model, we make the following assumptions:

1. The outcome of the $n_i$ strokes at each distance levels $x_i$ are *independent*. Each stroke in the group has probability $p_i$ of success.

2. The probability of success $p_i$ depends on the distance $x_i$ as follows:
    $$p_i =  \rm{sigm}(\alpha x + \beta)$$ 
    where 
    $$
    \rm{sigm}(z) = \frac{1}{1 + e^{-z}}.
    $$
3. The prior probability of the parameters 
$\theta \triangleq \begin{bmatrix}
\alpha \\
\beta
\end{bmatrix}$
is Gaussian: 
\begin{align}
\alpha &\sim N(\mu_\alpha, \sigma^2_\alpha), \qquad \mu_\alpha = 0, \sigma_\alpha=1\\
\beta &\sim N(\mu_\beta, \sigma^2_\beta), \qquad \mu_\beta=0, \sigma_\beta=1.
\end{align}
4. The outcomes of the 19 distance levels are independent of each other, given $\theta$.

## 1.1: Probabilistic model

* Derive and comment the full probabilistic model.

Putting together probabilistic assumptions 1-3, we obtain:

\begin{align*}
y_i | p_i &\sim  \mathrm{Binomial}(n_i, \rm{sigm}(\alpha x_i + \beta))\\
%p_i &= \rm{sigm}(\alpha + \beta x_i) \\
\alpha &\sim N(0, 1)\\
\beta &\sim N(0, 1).
\end{align*}

Furthermore, according to assumption 4:

$$P(y|\theta) = \prod_i P(y_i|\theta)$$

## 1.2: Maximum Likelihood estimation 

* Derive an analytical expression of the likelihood function $\mathcal{L}(\theta) = P(y|\theta)$.

The likelihood function $\mathcal{L}(\theta)$ is $P(y|\theta)$, seen as a function of $\theta$, with $y$ fixed to the observed outcome. <br/>Since the individual observations $y_i$ are independent, we have:

$$\mathcal{L}(\theta) = P(y|\theta) = \prod_{i=1}^N {{n_i}\choose{y_i}} \mathrm{sigm}(\alpha + \beta x_i)^{y_i} \cdot (1- \mathrm{sigm}(\alpha + \beta x_i))^{n_i - y_i}$$

* Derive an analytical expression of the log-likelihood function $\ell(\theta)$. 

$$\ell(\theta) = \log \mathcal{L}(\theta) = \sum_i {{n_i}\choose{y_i}} + \sum_i y_i \log \mathrm{sigm}(\alpha + \beta x_i) +  (n_i - y_i) \log (1- \mathrm{sigm}(\alpha + \beta x_i)).$$

The constant term $\sum_i {{n_i}\choose{y_i}}$ may be ignored.

* Write a Python function corresponding to the log-likelihood function $\ell(\theta)$, up to an additive term that does not depend on $\theta$.

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [5]:
def log_lik(alpha, beta):
    alpha = np.atleast_1d(alpha)[..., np.newaxis]  # useful to handle grid data
    beta = np.atleast_1d(beta)[..., np.newaxis]  # useful to handle grid data
    p = sigmoid(alpha * x + beta)
    # log_lik = y*np.log(gamma) + (n-y)*np.log(1-gamma)
    # nan_to_num handles the multiplication 0*np.inf and set it to 0, as required in our case...
    log_lik = np.nan_to_num(y * np.log(p), nan=0) + np.nan_to_num(
        (n - y) * np.log(1 - p), nan=0
    )
    return np.sum(log_lik, axis=-1)

* Visualize the log-likelihood $\ell(\theta)$ up to an additive term in 2D.

In [ ]:
dalpha = 0.01
dbeta = 0.01
ALPHA = np.arange(-6.0, -4.00, dalpha)
BETA = np.arange(1.5, 3.0, dbeta)
AA, BB = np.meshgrid(ALPHA, BETA, indexing="xy")

LOG_LIK = log_lik(AA, BB)
fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(AA, BB, LOG_LIK, cmap=cm.coolwarm, shading="auto")
fig.colorbar(c, ax=ax)
ax.set_title(f"Likelihood")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\beta$");

* Visualize the likelihood $\mathcal{L}(\theta)$ up to a multiplicative term in 2D. Explain the steps and comment the results.

HINT: you may (carefully) exponentiate the log-likelihood computed on the grid to solve the previous point.

In [ ]:
LIK_SAFE = np.exp(LOG_LIK - np.max(LOG_LIK))

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(AA, BB, LIK_SAFE, cmap=cm.coolwarm, shading="auto")
fig.colorbar(c, ax=ax)
ax.set_title(f"Likelihood")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\beta$");

* Compute the maximum likelihood (ML) estimate $\alpha^{\rm ml}, \beta^{\rm ml}$ of the parameters $\alpha, \beta$ through numerical optimizations. 

    Hints:
     * You may use the Python function `scipy.optimize.minimize`. 
     * You may look at the figures above to choose a good starting point for optimization 

In [8]:
from scipy.optimize import minimize

log_lik_theta = lambda theta: log_lik(theta[0], theta[1])
nll_theta = lambda theta: -log_lik_theta(theta)  # negative log-likelihood function.
res = minimize(nll_theta, x0=[4, -8])
theta_ml = res.x

* Visualize the likelihood together with the ML estimate. Comment the results.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(AA, BB, LIK_SAFE, cmap=cm.coolwarm, shading="auto")
plt.plot(theta_ml[0], theta_ml[1], "kx", label="ML")
fig.colorbar(c, ax=ax)
ax.set_title(f"Likelihood")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\beta$")
plt.legend();

## 1.2: Maximum A Posteriori Estimation

* Derive an analytical expression of the *unnormalized* posterior $f(\theta | y)$, i.e. up to a multiplicative term that does not depend on $\theta$.

Hint: exploit the already-obtained likelihood and the functional form of the Gaussian pdf.

$$f(\theta | y) = \frac{P(y | \theta) f(\theta)}{P(y)} \propto \mathcal{L}(\theta)
\exp\left(-\frac{1}{2} \frac{(\alpha - \mu_\alpha)^2}{\sigma^2_\alpha} \right ) 
\exp\left(-\frac{1}{2} \frac{(\beta - \mu_\beta)^2}{\sigma^2_\beta} \right ). $$ 

* Derive an analytical expression of the *unnormalized* log-posterior $\log f(\theta | y)$, i.e. up to an additive term that does not depend on $\theta$.

* Write a Python function corresponding to the  unnormalized log-posterior.

In [10]:
mu_alpha = 0.0
sigma_alpha = 1.0
mu_beta = 0.0
sigma_beta = 1.0

#prior_alpha = stats.norm(loc=mu_alpha, scale=sigma_alpha)
#prior_beta = stats.norm(loc=mu_beta, scale=sigma_beta)


def log_post_unscaled(alpha, beta):
    log_lik_val = log_lik(alpha, beta)
    # return lik_val * prior_alpha.pdf(alpha) * prior_beta.pdf(beta)
    return (
        log_lik_val
        - 0.5 * (alpha - mu_alpha) ** 2 / sigma_alpha**2
        - 0.5 * (beta - mu_beta) ** 2 / sigma_beta**2
    )

* Compute the maximum a posteriori (MAP) estimate $\alpha^{\rm MAP}, \beta^{\rm MAP}$.

In [11]:
minus_logpost = lambda theta: -log_post_unscaled(theta[0], theta[1])
res = minimize(minus_logpost, x0=[-5, 2])
theta_map = res.x

* Visualize the MAP and ML estimates in 2D, together with the unnormalized posterior. Comment the results.

In [ ]:
LOG_POST_UNSC = log_post_unscaled(AA, BB)
POST_UNSC = np.exp(LOG_POST_UNSC - np.max(LOG_POST_UNSC))

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(AA, BB, POST_UNSC, cmap=cm.coolwarm, shading="auto")
plt.plot(theta_map[0], theta_map[1], "kx", label="MAP")
plt.plot(theta_ml[0], theta_ml[1], "ko", label="ML")
fig.colorbar(c, ax=ax)
ax.set_title(f"Unnormalized posterior")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\beta$")
plt.legend();

## 1.3 Brute-force posterior estimation

* Compute (and visualize) a gridding approximation of the *normalized* posterior, i.e. with the correct normalization constant. Explain the steps.

We have:
    $$ \tilde f(\theta | y) = \mathcal{L}(\theta) \exp\left(-\frac{1}{2} 
(\theta - \mu)^{\top} \Sigma_0^{-1} (\theta - \mu)^{\top} \right) = Z f(\theta | y),$$
where $Z$ is the to-be-determined normalization constant and it must be chosen such that:
$$\iint f(\theta | y) d\alpha\; d\beta = 1.$$
Thus,
$$Z = \iint f(\theta | y) d\alpha\; d\beta.$$

The integral above is intractable, but a gridding approximation may be used. Using an equi-spaced gridding, a Riemann Sum approximation is:

$$Z \approx \Delta \alpha \Delta \beta \sum_i f(\theta_i | y),$$

where $\Delta \alpha$ and $\Delta \beta$ are the discretization steps of the 2D grid and $\theta_i$ are the grid points.

In [ ]:
# repreated from above for completeness
LOG_POST_UNSC = log_post_unscaled(AA, BB)
POST_UNSC = np.exp(LOG_POST_UNSC - np.max(LOG_POST_UNSC))

normalizing_factor = np.sum(POST_UNSC) * dalpha * dbeta
POST_SC = POST_UNSC / normalizing_factor


fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(AA, BB, POST_SC, cmap=cm.coolwarm, shading="auto")
plt.plot(theta_map[0], theta_map[1], "kx")
plt.plot(theta_ml[0], theta_ml[1], "ko")
fig.colorbar(c, ax=ax)
ax.set_title(f"Normalized posterior")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$\beta$");

* Using the grid-based approximation of the posterior, compute the posterior mean of $\alpha$ and $\beta$.

By definition, we have:

$$E[\theta] = \iint \theta p(\theta | y) d\alpha\; d\beta.$$

Using the grid-based approximation above:

$$E[\theta] = \Delta \alpha \Delta \beta \sum \theta_i p(\theta_i | y).$$

Software implementation below

In [ ]:
a_mean = np.sum(AA * POST_SC) * dalpha * dbeta
b_mean = np.sum(BB * POST_SC) * dalpha * dbeta
a_mean, b_mean

In [ ]:
x_range = np.linspace(0, 1, 100)
p_ml = sigmoid(theta_ml[0] * x_range + theta_ml[1])
p_map = sigmoid(theta_map[0] * x_range + theta_map[1])
plt.figure()
plt.plot(x_range, p_ml)
plt.plot(x_range, p_map)
plt.plot(x, frac, "*")

## 1.4 Monte Carlo estimation

* Obtain a sample-based approximation of the posterior $f(\theta | y)$ by implementing the Metropolis algorithm from scratch.
   * Run (at least) two chains
   * Diagnose the outcome of the different chains by overlapping the corresponding density plots. Is the algorithm sampling correctly?



In [16]:
def p_ratio_fun(alpha_propose, beta_propose, alpha_previous, beta_previous):
    log_p_previous = log_post_unscaled(alpha_previous, beta_previous)
    log_p_propose = log_post_unscaled(alpha_propose, beta_propose)
    log_p_ratio = log_p_propose - log_p_previous # log(p_prop/p_prev) = log(p_prop) - log(p_prev)
    p_ratio = np.exp(log_p_ratio)
    return p_ratio

In [ ]:
p_ratio_fun(alpha_propose = -5.0, alpha_previous = -5.1, beta_propose = 2.0, beta_previous = 2.1)

Let us run a Metropolis algorithm to sample from the posterior. The `p_ratio_fun` function is all we need!

In [18]:
N = 20_000 # number of Metropolis steps
warmup = 1_000
chains = 5
alpha_0 = mu_alpha # initial value for alpha. Another idea would be to start from the MAP...
beta_0 = mu_beta # initial value for alpha

alpha_step = alpha_0
beta_step = beta_0
sigma_prop_alpha = 0.2
sigma_prop_beta = 0.2

trace = []
for chain in range(chains):
    alphas = []
    betas = []
    for idx in range(N):
        alphas.append(alpha_step)
        betas.append(beta_step)

        alpha_prop = alpha_step + sigma_prop_alpha * np.random.randn()
        beta_prop = beta_step + sigma_prop_beta * np.random.randn()
    
        p_ratio = p_ratio_fun(alpha_prop, beta_prop, alpha_step, beta_step)
        accept_prob = np.minimum(1.0, p_ratio)
        accept = (np.random.rand() < accept_prob)
        
        if accept:
            alpha_step = alpha_prop
            beta_step = beta_prop

    alphas = np.stack(alphas)
    betas = np.stack(betas)
    trace_chain = np.c_[alphas, betas]
    trace.append(trace_chain)
trace = np.stack(trace, axis=0)


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
plt.suptitle("Trace")
for var in range(2):
    for chain in range(chains):
        #ax[var, 0].hist(trace[chain, warmup:, var], density=True, alpha=0.4)
        sns.kdeplot(trace[chain, warmup:, var], ax=ax[var, 0])
        ax[var, 1].plot(trace[chain, :, var])
    ax[var, 1].axvline(x=warmup, color="red")


In [ ]:
traces = np.concatenate(trace[:, warmup:, :])
x_range = np.linspace(0, 1, 100)
p_samples = sigmoid(np.expand_dims(traces[:, 0], axis=-1) * x_range + np.expand_dims(traces[:, 1], axis=-1))
plt.figure()
plt.plot(x_range, p_samples[:1000, :].T, "b", alpha=1.0)
plt.plot(x_range, p_map, "k")
plt.plot(x, frac, "*")
#plt.plot(x, prop, "*")

* Compare the results of gridding and Metropolis

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].hist2d(x=trace_chain[warmup:, 0], y=trace_chain[warmup:, 1], bins=100, cmap=plt.cm.BuPu)
ax[0].set_xlim([-6, -4]);
ax[0].set_ylim([1.5, 3]);
ax[0].contour(AA, BB, POST_SC); #, levels=[5, 15,  95]); # levels=[5, 15, 25, 35, 45, 55, 65, 75, 85, 95])
c = ax[1].pcolormesh(AA, BB, POST_SC, cmap=cm.coolwarm, shading='auto')